# Present Wrapping Problem

In [2]:
%load_ext autoreload
%load_ext iminizinc
%matplotlib inline
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from z3 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


## Functions

In [3]:
def read_input(path):
    file = open(path,"r").readlines()
    w_paper , h_paper = tuple(map(int, file[0].rstrip("\n").split(" ")))
    n = int(file[1].rstrip("\n"))
    presents = []
    for i in range(2, n + 2):
        presents.append(list(map(int, file[i].rstrip("\n").split(" "))))
    return w_paper, h_paper, n, presents

In [266]:
def show_solutions(w_paper, h_paper, n, presents, solutions):
    colours = np.random.rand(n, 3)
    if not isinstance(solutions, list):
        solutions = [solutions]
    for x in solutions:
        bl_corners = x['bl_corners'] if 'bl_corners' in x else x
        show_solution(w_paper, h_paper, n, presents, bl_corners, colours)

def show_solution(w_paper, h_paper, n,presents, bl_corners, colours=None):
    if colours is None:
        colours = np.random.rand(n, 3)
    fig, ax = plt.subplots()
    for i in range(n):
        ax.add_patch(patches.Rectangle(
            bl_corners[i],
            presents[i][0],
            presents[i][1],
            facecolor=colours[i]
        ))
    ax.set_xlim(0, w_paper)
    ax.set_ylim(0, h_paper)
    plt.show()
    print(f"Solution: {bl_corners}")

In [144]:
def order_presents(presents, method='height'):
    to_sort = (
        [e[0] * e[1] for e in presents] if method == 'area'
        else [e[1] for e in presents] if method == 'height'
        else [e[0] for e in presents] if method == 'width'
        else None
    )
    sorted_indices = sorted(range(len(to_sort)), key=lambda k: to_sort[k], reverse=True)
    presents = [presents[i] for i in sorted_indices]
    return presents, sorted_indices

## Input

In [334]:
min_instance, max_instance = 8, 20
#instance = np.random.randint(min_instance, high=max_instance + 1)
instance = 
path = f"instances/{instance}x{instance}.txt"
w_paper, h_paper, n, presents = read_input(path)
unordered_presents = list(presents)
presents, sorted_indices = order_presents(presents, method='height')
bl_corners = []
print(f"Paper dimensions: {instance}x{instance}")
print(f"Total area: {sum(areas)} / {instance * instance}")
print(f"Presents dimensions: {presents}")

Paper dimensions: 8x8
Total area: 1296 / 64
Presents dimensions: [[3, 5], [5, 5], [3, 3], [5, 3]]


## MiniZinc model

### First model 

In [324]:
%%mzn_model pwp-v1
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
array[1..n, 1..2] of var 0..max(w_paper, h_paper) - 1: bl_corners;
array[1..n, 1..2] of int: presents;
constraint forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] <= w_paper);
constraint forall(i in 1..n) (bl_corners[i, 2] + presents[i, 2] <= h_paper);
constraint forall(i, j in 1..n where j > i) (
    max(bl_corners[i, 1], bl_corners[j, 1]) >= min(bl_corners[i, 1] + presents[i, 1], bl_corners[j, 1] + presents[j, 1]) \/
    max(bl_corners[i, 2], bl_corners[j, 2]) >= min(bl_corners[i, 2] + presents[i, 2], bl_corners[j, 2] + presents[j, 2])
);
solve satisfy;

In [325]:
%minizinc -s pwp-v1

KeyboardInterrupt: 

### Second model

In [ ]:
%%mzn_model pwp-v2
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
array[1..n, 1..2] of int: presents;
int: max_dim = max(w_paper, h_paper);
int: min_present_dim = min([min(presents[i, 1], presents[i, 2]) | i in 1..n]);

function var int: coord_to_value(var int: x, var int: y) = x + y * (max_dim + 1);
function var int: x_overlap(int: i, int: j) = 
    max(0, min(bl_corners[i, 1] + presents[i, 1], bl_corners[j, 1] + presents[j, 1]) - max(bl_corners[i, 1], bl_corners[j, 1]));
function var int: y_overlap(int: i, int: j) = 
    max(0, min(bl_corners[i, 2] + presents[i, 2], bl_corners[j, 2] + presents[j, 2]) - max(bl_corners[i, 2], bl_corners[j, 2]));
function var int: overlap_area(int: i, int: j) = x_overlap(i, j) * y_overlap(i, j);

array[1..n, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[1..n] of var 0..max_dim * max_dim: bl_corners_values;

predicate values_channeling(int: i) = 
    forall(i in 1..n) (bl_corners_values[i] == coord_to_value(bl_corners[i, 1], bl_corners[i, 2]));
constraint forall(i in 1..n) (values_channeling(i));

constraint alldifferent(bl_corners_values);
constraint count_eq(bl_corners_values, 0, 1);
constraint forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] <= w_paper);
constraint forall(i in 1..n) (bl_corners[i, 2] + presents[i, 2] <= h_paper);
constraint forall(i, j in 1..n where j > i) (overlap_area(i, j) == 0);
solve satisfy;

In [ ]:
%minizinc -s pwp-v2

### Third model

In [ ]:
%%mzn_model pwp-v3
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
int: max_dim = max(w_paper, h_paper);
array[1..n, 1..2] of 1..max_dim: presents;
array[1..n] of 1..max_dim: presents_xs = [presents[i, 1] | i in 1..n];
array[1..n] of 1..max_dim: presents_ys = [presents[i, 2] | i in 1..n];
int: min_present_dim = min([min(presents[i, 1], presents[i, 2]) | i in 1..n]);

function var int: coord_to_value(var int: x, var int: y) = x + y * (max_dim + 1);

array[1..n, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[1..n, 1..2] of var 1..max_dim: tr_corners;
array[1..n] of var 0..max_dim * max_dim: bl_corners_values;
array[1..n] of var 0..w_paper - min_present_dim: bl_corners_xs;
array[1..n] of var 0..h_paper - min_present_dim: bl_corners_ys;

predicate values_channeling(int: i) = 
    forall(i in 1..n) (bl_corners_values[i] == coord_to_value(bl_corners[i, 1], bl_corners[i, 2]));
constraint forall(i in 1..n) (values_channeling(i));

predicate bl_corners_channeling(int: i) = 
    forall(i in 1..n) (bl_corners[i, 1] == bl_corners_xs[i] /\ bl_corners[i, 2] == bl_corners_ys[i]);
constraint forall(i in 1..n) (bl_corners_channeling(i));

predicate tr_corners_channeling(int: i) = 
    forall(i in 1..n) (bl_corners[i, 1] + presents[i, 1] == tr_corners[i, 1] /\ bl_corners[i, 2] + presents[i, 2] == tr_corners[i, 2]);
constraint forall(i in 1..n) (tr_corners_channeling(i));

constraint alldifferent(bl_corners_values);
constraint count_eq(bl_corners_values, 0, 1);
constraint diffn_k(bl_corners, presents);
constraint cumulative(bl_corners_xs, presents_xs, presents_ys, h_paper);
constraint cumulative(bl_corners_ys, presents_ys, presents_xs, w_paper);
solve satisfy;

In [ ]:
%minizinc -s pwp-v3

### Fourth model

In [335]:
%%mzn_model pwp-v4
include "globals.mzn";
int: w_paper;
int: h_paper;
int: n;
int: max_dim = max(w_paper, h_paper);
int: total_area = w_paper * h_paper;
array[1..n, 1..2] of 1..max_dim: presents;
array[1..n] of 1..max_dim: presents_xs = [presents[i, 1] | i in 1..n];
array[1..n] of 1..max_dim: presents_ys = [presents[i, 2] | i in 1..n];
array[1..n] of 1..max_dim * max_dim: areas = [presents_xs[i] * presents_ys[i] | i in 1..n];
int: min_present_dim = min([min(presents[i, 1], presents[i, 2]) | i in 1..n]);

array[1..n, 1..2] of var 0..max_dim - min_present_dim: bl_corners;
array[1..n, 1..2] of var 1..max_dim: tr_corners;
array[1..w_paper, 1..h_paper] of var bool: rectangles;
array[1..n] of var 0..max_dim * max_dim: bl_corners_values;
array[1..n] of var 0..w_paper - min_present_dim: bl_corners_xs;
array[1..n] of var 0..h_paper - min_present_dim: bl_corners_ys;

function var int: coord_to_value(var int: x, var int: y) = x + y * (max_dim + 1);
function var bool: coord_to_rect(var int: x, var int: y, int: i) =
    x + 1 <= tr_corners[i, 1] /\ x + 1 > bl_corners[i, 1] /\ 
    y + 1 > bl_corners[i, 2] /\ y + 1 <= tr_corners[i, 2];
function var int: rotate_by_180(int: i) = coord_to_value(w_paper - bl_corners_xs[i] - presents_xs[i], h_paper - bl_corners_ys[i] - presents_ys[i]);

predicate bl_corners_channeling(int: i) = bl_corners_xs[i] == bl_corners[i, 1] /\ bl_corners_ys[i] == bl_corners[i, 2];
constraint forall(i in 1..n) (bl_corners_channeling(i));

predicate values_channeling(int: i) = bl_corners_values[i] == coord_to_value(bl_corners_xs[i], bl_corners_ys[i]);
constraint forall(i in 1..n) (values_channeling(i));

predicate tr_corners_channeling(int: i) = tr_corners[i, 1] == bl_corners_xs[i] + presents_xs[i] /\ tr_corners[i, 2] == bl_corners_ys[i] + presents_ys[i];
constraint forall(i in 1..n) (tr_corners_channeling(i));


%predicate rectangles_channeling(int: x, int: y) = exists(i in 1..n) ((rectangles[x, y] == i) <-> coord_to_rect(x, y, i));
%constraint forall(x in 1..w_paper, y in 1..h_paper) (rectangles_channeling(x, y));

%constraint forall(i in 1..n) (count_eq(array1d(rectangles), i, areas[i]));
%constraint symmetry_breaking_constraint(lex_lesseq(array1d(rectangles), reverse(array1d(rectangles))));

%constraint symmetry_breaking_constraint(lex_lesseq(bl_corners_values, [rotate_by_180(i) | i in 1..n]));

%constraint lex_lesseq(bl_corners_values, reverse(bl_corners_values));
%constraint bl_corners_xs[1] <= w_paper / 2 /\ bl_corners_ys[1] <= h_paper / 2;
constraint sum(i in 1..n) ((tr_corners[i, 1] - bl_corners[i, 1]) * (tr_corners[i, 2] - bl_corners[i, 2])) == total_area;
constraint alldifferent(bl_corners_values);
constraint count_eq(bl_corners_values, 0, 1);
constraint diffn_k(bl_corners, presents);
constraint cumulative(bl_corners_xs, presents_xs, presents_ys, h_paper);
constraint cumulative(bl_corners_ys, presents_ys, presents_xs, w_paper);
%solve :: int_search(bl_corners_values, input_order, indomain_min) satisfy;
solve satisfy;

In [336]:
solutions = %minizinc -a -s pwp-v4

(1,1,1)(1,1,2)(1,1,3)(1,1,4)(1,2,1)(1,2,2)(1,2,3)(1,2,4)(1,3,1)(1,3,2)(1,3,3)(1,3,4)(1,4,1)(1,4,2)(1,4,3)(1,4,4)(1,5,1)(1,5,2)(1,5,3)(1,5,4)(1,6,1)(1,6,2)(1,6,3)(1,6,4)(2,1,1)(2,1,2)(2,1,3)(2,1,4)(2,2,1)(2,2,2)(2,2,3)(2,2,4)(2,3,1)(2,3,2)(2,3,3)(2,3,4)(2,4,1)(2,4,2)(2,4,3)(2,4,4)(2,5,1)(2,5,2)(2,5,3)(2,5,4)(2,6,1)(2,6,2)(2,6,3)(2,6,4)(3,1,1)(3,1,2)(3,1,3)(3,1,4)(3,2,1)(3,2,2)(3,2,3)(3,2,4)(3,3,1)(3,3,2)(3,3,3)(3,3,4)(3,4,1)(3,4,2)(3,4,3)(3,4,4)(3,5,1)(3,5,2)(3,5,3)(3,5,4)(3,6,1)(3,6,2)(3,6,3)(3,6,4)(4,1,1)(4,1,2)(4,1,3)(4,1,4)(4,2,1)(4,2,2)(4,2,3)(4,2,4)(4,3,1)(4,3,2)(4,3,3)(4,3,4)(4,4,1)(4,4,2)(4,4,3)(4,4,4)(4,5,1)(4,5,2)(4,5,3)(4,5,4)(4,6,1)(4,6,2)(4,6,3)(4,6,4)(5,1,1)(5,1,2)(5,1,3)(5,1,4)(5,2,1)(5,2,2)(5,2,3)(5,2,4)(5,3,1)(5,3,2)(5,3,3)(5,3,4)(5,4,1)(5,4,2)(5,4,3)(5,4,4)(5,5,1)(5,5,2)(5,5,3)(5,5,4)(5,6,1)(5,6,2)(5,6,3)(5,6,4)(6,1,1)(6,1,2)(6,1,3)(6,1,4)(6,2,1)(6,2,2)(6,2,3)(6,2,4)(6,3,1)(6,3,2)(6,3,3)(6,3,4)(6,4,1)(6,4,2)(6,4,3)(6,4,4)(6,5,1)(6,5,2)(6,5,3)(6,5,4)(6,6,1)(6,6,2)(6,6,3

In [314]:
show_solutions(w_paper, h_paper, n, presents, solutions)

TypeError: argument of type 'NoneType' is not iterable

## SMT model

In [119]:
def z3max(x,y):
    return If(x > y, x, y)

def z3min(x,y):
    return If(x < y, x, y)

bl_corners= [ [ Int("c_%s_%s" % (i, j)) for j in range(2) ] for i in range(n) ]
domain = [And(0<=bl_corners[i][0],bl_corners[i][0]<w_paper,0<=bl_corners[i][1],bl_corners[i][1]< h_paper) for i in range(n)]
overflow = [And(bl_corners[i][0]+presents[i][0]<=w_paper,bl_corners[i][1]+presents[i][1]<=h_paper) for i in range(n)]
pp(bl_corners)
intersection = [ And(
    Or(
        z3max(bl_corners[i][0],bl_corners[j][0])>=z3min(bl_corners[i][0]+presents[i][0],bl_corners[j][0]+presents[j][0]),
        z3max(bl_corners[i][1],bl_corners[j][1])>=z3min(bl_corners[i][1]+presents[i][1],bl_corners[j][1]+presents[j][1])
     )) for i in range(n) for j in range(i+1,n) ]

[[c_0_0, c_0_1],
 [c_1_0, c_1_1],
 [c_2_0, c_2_1],
 [c_3_0, c_3_1],
 [c_4_0, c_4_1],
 [c_5_0, c_5_1],
 [c_6_0, c_6_1],
 [c_7_0, c_7_1],
 [c_8_0, c_8_1],
 [c_9_0, c_9_1],
 [c_10_0, c_10_1],
 [c_11_0, c_11_1],
 [c_12_0, c_12_1],
 [c_13_0, c_13_1],
 [c_14_0, c_14_1],
 [c_15_0, c_15_1],
 [c_16_0, c_16_1],
 [c_17_0, c_17_1],
 [c_18_0, c_18_1],
 [c_19_0, c_19_1],
 [c_20_0, c_20_1],
 [c_21_0, c_21_1]]


In [121]:
%%time
s = Solver()
s.add(domain + overflow + intersection)
s.check()
sol = s.model()
solution=[[sol[bl_corners[i][0]], sol[bl_corners[i][1]]] for i in range(n)]
solution

CPU times: user 15min 5s, sys: 1.69 s, total: 15min 7s
Wall time: 15min 8s


[[18, 12],
 [8, 31],
 [18, 15],
 [15, 29],
 [18, 28],
 [18, 20],
 [8, 22],
 [11, 0],
 [18, 0],
 [15, 10],
 [8, 0],
 [0, 0],
 [14, 0],
 [4, 0],
 [14, 4],
 [4, 5],
 [11, 10],
 [0, 23],
 [11, 18],
 [0, 3],
 [4, 12],
 [21, 0]]